 # 4D-OBC merge
 
This notebook explains how to use the merge-function of the py4dgeo-packag which automatically merges similar 4D-OBCs to one.  

Implemented in: `py4dgeo.merge`

**Author(s)** 

Scharnagl, Robin (Technische Universität München)

**Related publication**

 Ulm M, Elias M,  Eltner A, Lotsari E & Anders K. (2025). Automated change detection in photogrammetric 4D point clouds – transferability and extension of 4D objects‑by‑change 
for monitoring riverbank dynamics using low‑cost cameras. *Applied Geomatics*, https://doi.org/10.1007/s12518-025-00623-9   


## **Method description**
This section should provide a detailed technical explanation of the method:

- **Theoretical foundation**: Due to uncorrelated noise, the extraction of 4D-OBCs tends to oversegment, which leads to multiple 4D-OBCs describing one process. This function merges 4D-OBCs that overlap in time and space, to create just one object.

- **Algorithm**: The algorithm first checks the change direction of each object. It then calculates the spatial and temporal overlap of the OBCs. If the OBC's changes point in the same direction, and their temporal and spatial overlap exceeds a user-defined threshold, their link is stored. In the last step, these get fused to one single 4D-OBC

- **Key parameters**: The parameters that the user can define are the following:
    - `time_threshold` (float, 0 - 1): threshold for temporal overlap, 1: complete overlap, 0: no overlap
    - `spatial_threshold`(float, 0 - 1): thershold for spatial overlap, 1: complete overlap, 0: no overlap


First, we download the dataset:

In [1]:
# Load Required Package
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pooch
import os 
from datetime import datetime
import py4dgeo 


# Load Dataset
# Download data from zenodo and set path to point cloud folder
p = pooch.Pooch(base_url="doi:10.5281/zenodo.10003574/", path=pooch.os_cache("py4dgeo"))
p.load_registry_from_doi()
p.fetch("module3.zip", processor=pooch.Unzip(), progressbar=True)
pc_dir = p.path / "module3.zip.unzip" / "module3" / "schneeferner" / "pointclouds"


100%|#############################################| 2.13G/2.13G [00:00<?, ?B/s]
Unzipping contents of 'C:\Users\schar\AppData\Local\py4dgeo\py4dgeo\Cache\module3.zip' to 'C:\Users\schar\AppData\Local\py4dgeo\py4dgeo\Cache\module3.zip.unzip'


We then create a SpatiotemporalAnalysis-object from the given dataset.

In [2]:
# list of point clouds (time series)
pc_list = os.listdir(pc_dir)
pc_list[:5] # print the first elements


timestamps = []
for f in pc_list:
    if not f.endswith('.laz'):
        continue

    # get the timestamp from the file name
    timestamp_str = '_'.join(f.split('.')[0].split('_')[1:]) # yields YYMMDD_hhmmss

    # convert string to datetime object
    timestamp = datetime.strptime(timestamp_str, '%y%m%d_%H%M%S')
    timestamps.append(timestamp)

timestamps[:5]

analysis = py4dgeo.SpatiotemporalAnalysis(f'{pc_dir}\schneeferner.zip', force=True)
#%%
# specify the reference epoch
reference_epoch_file = os.path.join(pc_dir, pc_list[0])

# read the reference epoch and set the timestamp
reference_epoch = py4dgeo.read_from_las(reference_epoch_file)
reference_epoch.timestamp = timestamps[0]

# set the reference epoch in the spatiotemporal analysis object
analysis.reference_epoch = reference_epoch

<>:23: SyntaxWarning: invalid escape sequence '\s'
<>:23: SyntaxWarning: invalid escape sequence '\s'
C:\Users\schar\AppData\Local\Temp\ipykernel_55952\1910519347.py:23: SyntaxWarning: invalid escape sequence '\s'
  analysis = py4dgeo.SpatiotemporalAnalysis(f'{pc_dir}\schneeferner.zip', force=True)


[2025-10-22 11:54:30][INFO] Creating analysis file C:\Users\schar\AppData\Local\py4dgeo\py4dgeo\Cache\module3.zip.unzip\module3\schneeferner\pointclouds\schneeferner.zip
[2025-10-22 11:54:30][INFO] Reading point cloud from file 'C:\Users\schar\AppData\Local\py4dgeo\py4dgeo\Cache\module3.zip.unzip\module3\schneeferner\pointclouds\schneeferner_180418_120027.laz'
[2025-10-22 11:54:30][INFO] Building KDTree structure with leaf parameter 10
[2025-10-22 11:54:30][INFO] Saving epoch to file 'C:\Users\schar\AppData\Local\Temp\tmpu6j749zf\reference_epoch.zip'
[2025-10-22 11:54:31][INFO] Saving a file without normals.


With the SpatiotemporalAnalysis-object, we can simply add the merge-function and define the parameters to the users-preference. 

In [3]:
# Run the Method
merged = analysis.merge(time_threshold=0.7, spatial_threshold=0.1)

# Plot a merged object of change
merged[0].plot()

AttributeError: 'SpatiotemporalAnalysis' object has no attribute 'merge'

## References
-  Ulm M, Elias M,  Eltner A, Lotsari E & Anders K. (2025). Automated change detection in photogrammetric 4D point clouds – transferability and extension of 4D objects‑by‑change 
for monitoring riverbank dynamics using low‑cost cameras. *Applied Geomatics*, pages. https://doi.org/10.1007/s12518-025-00623-9   